# Network Traffic Classification using Machine Learning Techniques

# Overview

Develop classification models using Python programming to analyze a network-related dataset. 

The primary goal is to explore the dataset, preprocess it, create and evaluate different classification models, and report your findings. 

This assignment will enhance your understanding of machine learning techniques, data preprocessing, and model evaluation while applying them to a practical problem related to network security.

# Dataset

This is a real-world dataset created by collecting network data from Universidad Del Cauca, Popayn, Colombia over six days (April 26, 27, 28 and May 9, 11 and 15) of 2017 using multiple packet capturing tools and data extracting tools. 

This dataset is consisting of 3,577,296 instances and 87 features and originally designed for application classification. Each row represents a traffic flow from a source to a destination and each column represents features of the traffic data.

This dataset is downloaded from Kaggle "IP Network Traffic Flows, Labeled with 75 Apps."

# Purpose

# Literature Review

# Assumption Made

# Origin of CICFlowMeter

https://www.unb.ca/cic/research/applications.html#CICFlowMeter

https://www.kaggle.com/datasets/jsrojas/ip-network-traffic-flows-labeled-with-87-apps 

https://www.ntop.org/products/traffic-analysis/ntop/

# Environment Setup

In [1]:
!pip install --upgrade pip
!pip install setuptools -U
!pip install pandas -U
!pip install -U scikit-learn
!pip install kagglehub -U

!pip install matplotlib -U
!pip install seaborn -U

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 26.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 29.1 MB/s eta 0:00:00
Defaulting to user installation because normal site-packages is not writeable


In [2]:
%pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 37.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install fireducks

In [5]:
import kagglehub
import pandas
import numpy

import matplotlib
import seaborn
import fireducks.pandas

# Data Retrieval

Retrieving data using kagglehub package to simplify the data retrieval process

In [2]:
path = kagglehub.dataset_download("jsrojas/ip-network-traffic-flows-labeled-with-87-apps")

In [3]:
print(path)

/home/vscode/.cache/kagglehub/datasets/jsrojas/ip-network-traffic-flows-labeled-with-87-apps/versions/1


# Data Loading into DataFrame

Attempting to load the data into pandas dataframe for the data exploration

In [10]:
network_traffic_analysis_dataframe: fireducks.pandas.frame.DataFrame = fireducks.pandas.read_csv("Dataset-Unicauca-Version2-87Atts.csv")

In [11]:
network_traffic_analysis_dataframe

,Flow.ID,Source.IP,Source.Port,Destination.IP,Destination.Port,Protocol,Timestamp,Flow.Duration,Total.Fwd.Packets,Total.Backward.Packets,...,Active.Std,Active.Max,Active.Min,Idle.Mean,Idle.Std,Idle.Max,Idle.Min,Label,L7Protocol,ProtocolName
0,172.19.1.46-10.200.7.7-52422-3128-6,172.19.1.46,52422,10.200.7.7,3128,6,26/04/201711:11:17,45523,22,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,131,HTTP_PROXY
1,172.19.1.46-10.200.7.7-52422-3128-6,10.200.7.7,3128,172.19.1.46,52422,6,26/04/201711:11:17,1,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,131,HTTP_PROXY
2,10.200.7.217-50.31.185.39-38848-80-6,50.31.185.39,80,10.200.7.217,38848,6,26/04/201711:11:17,1,3,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,7,HTTP
3,10.200.7.217-50.31.185.39-38848-80-6,50.31.185.39,80,10.200.7.217,38848,6,26/04/201711:11:17,217,1,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,7,HTTP
4,192.168.72.43-10.200.7.7-55961-3128-6,192.168.72.43,55961,10.200.7.7,3128,6,26/04/201711:11:17,78068,5,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,131,HTTP_PROXY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3577291,10.200.7.199-98.138.79.73-42135-443-6,98.138.79.73,443,10.200.7.199,42135,6,15/05/201705:43:40,2290821,5,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,91,SSL
3577292,10.200.7.217-98.138.79.73-51546-443-6,98.138.79.73,443,10.200.7.217,51546,6,15/05/201705:46:10,24,5,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,91,SSL
3577293,10.200.7.218-98.138.79.73-44366-443-6,98.138.79.73,443,10.200.7.218,44366,6,15/05/201705:45:39,2591653,6,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,91,SSL
3577294,10.200.7.195-98.138.79.73-52341-443-6,98.138.79.73,443,10.200.7.195,52341,6,15/05/201705:45:59,2622421,4,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,91,SSL


# Data Exploration

### Data Type for all columns in the dataset

In [25]:
print(network_traffic_analysis_dataframe.dtypes[:20])

Source.IP                       object
Source.Port                      int64
Destination.IP                  object
Destination.Port                 int64
Protocol                         int64
Timestamp                       object
Flow.Duration                    int64
Total.Fwd.Packets                int64
Total.Backward.Packets           int64
Total.Length.of.Fwd.Packets      int64
Total.Length.of.Bwd.Packets    float64
Fwd.Packet.Length.Max            int64
Fwd.Packet.Length.Min            int64
Fwd.Packet.Length.Mean         float64
Fwd.Packet.Length.Std          float64
Bwd.Packet.Length.Max            int64
Bwd.Packet.Length.Min            int64
Bwd.Packet.Length.Mean         float64
Bwd.Packet.Length.Std          float64
Flow.Bytes.s                   float64
dtype: object


In [26]:
print(network_traffic_analysis_dataframe.dtypes[21:40])

Flow.IAT.Mean        float64
Flow.IAT.Std         float64
Flow.IAT.Max         float64
Flow.IAT.Min           int64
Fwd.IAT.Total        float64
Fwd.IAT.Mean         float64
Fwd.IAT.Std          float64
Fwd.IAT.Max          float64
Fwd.IAT.Min          float64
Bwd.IAT.Total        float64
Bwd.IAT.Mean         float64
Bwd.IAT.Std          float64
Bwd.IAT.Max          float64
Bwd.IAT.Min          float64
Fwd.PSH.Flags          int64
Bwd.PSH.Flags          int64
Fwd.URG.Flags          int64
Bwd.URG.Flags          int64
Fwd.Header.Length      int64
dtype: object


In [27]:
print(network_traffic_analysis_dataframe.dtypes[41:60])

Fwd.Packets.s             float64
Bwd.Packets.s             float64
Min.Packet.Length           int64
Max.Packet.Length           int64
Packet.Length.Mean        float64
Packet.Length.Std         float64
Packet.Length.Variance    float64
FIN.Flag.Count              int64
SYN.Flag.Count              int64
RST.Flag.Count              int64
PSH.Flag.Count              int64
ACK.Flag.Count              int64
URG.Flag.Count              int64
CWE.Flag.Count              int64
ECE.Flag.Count              int64
Down.Up.Ratio               int64
Average.Packet.Size       float64
Avg.Fwd.Segment.Size      float64
Avg.Bwd.Segment.Size      float64
dtype: object


In [28]:
print(network_traffic_analysis_dataframe.dtypes[71:])

Init_Win_bytes_forward       int64
Init_Win_bytes_backward      int64
act_data_pkt_fwd             int64
min_seg_size_forward         int64
Active.Mean                float64
Active.Std                 float64
Active.Max                 float64
Active.Min                 float64
Idle.Mean                  float64
Idle.Std                   float64
Idle.Max                   float64
Idle.Min                   float64
Label                       object
L7Protocol                   int64
ProtocolName                object
dtype: object


### Feature and its description

- Flow duration		
    - Duration of the flow in Microsecond
- total Fwd Packet		
    - Total packets in the forward direction
- total Bwd packets		
    - Total packets in the backward direction
- total Length of Fwd Packet	
    - Total size of packet in forward direction
- total Length of Bwd Packet	
    - Total size of packet in backward direction
- Fwd Packet Length Min 		
    - Minimum size of packet in forward direction
- Fwd Packet Length Max 		
    - Maximum size of packet in forward direction
- Fwd Packet Length Mean		
    - Mean size of packet in forward direction
- Fwd Packet Length Std		
    - Standard deviation size of packet in forward direction
- Bwd Packet Length Min		
    - Minimum size of packet in backward direction
- Bwd Packet Length Max		
    - Maximum size of packet in backward direction
- Bwd Packet Length Mean		
    - Mean size of packet in backward direction
- Bwd Packet Length Std		
    - Standard deviation size of packet in backward direction
- Flow Byte/s			
    - Number of flow packets per second
- Flow Packets/s			
    - Number of flow bytes per second 
- Flow IAT Mean			
    - Mean time between two packets sent in the flow
- Flow IAT Std			
    - Standard deviation time between two packets sent in the flow
- Flow IAT Max			
    - Maximum time between two packets sent in the flow
- Flow IAT Min			
    - Minimum time between two packets sent in the flow
- Fwd IAT Min			
    - Minimum time between two packets sent in the forward direction
- Fwd IAT Max			
    - Maximum time between two packets sent in the forward direction
- Fwd IAT Mean			
    - Mean time between two packets sent in the forward direction
- Fwd IAT Std			
    - Standard deviation time between two packets sent in the forward direction
- Fwd IAT Total   		
    - Total time between two packets sent in the forward direction
- Bwd IAT Min			
    - Minimum time between two packets sent in the backward direction
- Bwd IAT Max			
    - Maximum time between two packets sent in the backward direction
- Bwd IAT Mean			
    - Mean time between two packets sent in the backward direction
- Bwd IAT Std			
    - Standard deviation time between two packets sent in the backward direction
- Bwd IAT Total			
    - Total time between two packets sent in the backward direction
- Fwd PSH flag			
    - Number of times the PSH flag was set in packets travelling in the forward direction (0 for UDP)
- Bwd PSH Flag			
    - Number of times the PSH flag was set in packets travelling in the backward direction (0 for UDP)
- Fwd URG Flag			
    - Number of times the URG flag was set in packets travelling in the forward direction (0 for UDP)
- Bwd URG Flag			
    - Number of times the URG flag was set in packets travelling in the backward direction (0 for UDP)
- Fwd Header Length		
    - Total bytes used for headers in the forward direction
- Bwd Header Length		
    - Total bytes used for headers in the backward direction
- FWD Packets/s			
    - Number of forward packets per second
- Bwd Packets/s			
    - Number of backward packets per second
- Min Packet Length 		
    - Minimum length of a packet
- Max Packet Length 		
    - Maximum length of a packet
- Packet Length Mean 		
    - Mean length of a packet
- Packet Length Std		
    - Standard deviation length of a packet
- Packet Length Variance  	
    - Variance length of a packet
- FIN Flag Count 			
    - Number of packets with FIN
- SYN Flag Count 			
    - Number of packets with SYN
- RST Flag Count 			
    - Number of packets with RST
- PSH Flag Count 			
    - Number of packets with PUSH
- ACK Flag Count 			
    - Number of packets with ACK
- URG Flag Count 			
    - Number of packets with URG
- CWR Flag Count 			
    - Number of packets with CWE
- ECE Flag Count 			
    - Number of packets with ECE
- down/Up Ratio			
    - Download and upload ratio
- Average Packet Size 		
    - Average size of packet
- Avg Fwd Segment Size 		
    - Average size observed in the forward direction
- AVG Bwd Segment Size 		
    - Average number of bytes bulk rate in the backward direction
- Fwd Header Length		
    - Length of the forward packet header
- Fwd Avg Bytes/Bulk		
    - Average number of bytes bulk rate in the forward direction
- Fwd AVG Packet/Bulk 		
    - Average number of packets bulk rate in the forward direction
- Fwd AVG Bulk Rate 		
    - Average number of bulk rate in the forward direction
- Bwd Avg Bytes/Bulk		
    - Average number of bytes bulk rate in the backward direction
- Bwd AVG Packet/Bulk 		
    - Average number of packets bulk rate in the backward direction
- Bwd AVG Bulk Rate 		
    - Average number of bulk rate in the backward direction
- Subflow Fwd Packets		
    - The average number of packets in a sub flow in the forward direction
- Subflow Fwd Bytes		
    - The average number of bytes in a sub flow in the forward direction
- Subflow Bwd Packets		
    - The average number of packets in a sub flow in the backward direction
- Subflow Bwd Bytes		
    - The average number of bytes in a sub flow in the backward direction
- Init_Win_bytes_forward		
    - The total number of bytes sent in initial window in the forward direction
- Init_Win_bytes_backward		
    - The total number of bytes sent in initial window in the backward direction
- Act_data_pkt_forward		
    - Count of packets with at least 1 byte of TCP data payload in the forward direction
- min_seg_size_forward		
    - Minimum segment size observed in the forward direction
- Active Min			
    - Minimum time a flow was active before becoming idle
- Active Mean			
    - Mean time a flow was active before becoming idle
- Active Max			
    - Maximum time a flow was active before becoming idle
- Active Std			
    - Standard deviation time a flow was active before becoming idle
- Idle Min			
    - Minimum time a flow was idle before becoming active
- Idle Mean			
    - Mean time a flow was idle before becoming active
- Idle Max			
    - Maximum time a flow was idle before becoming active
- Idle Std			
    - Standard deviation time a flow was idle before becoming active

### Dataset Information

In [32]:
network_traffic_analysis_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3577296 entries, 0 to 3577295
Data columns (total 86 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   Source.IP                    object 
 1   Source.Port                  int64  
 2   Destination.IP               object 
 3   Destination.Port             int64  
 4   Protocol                     int64  
 5   Timestamp                    object 
 6   Flow.Duration                int64  
 7   Total.Fwd.Packets            int64  
 8   Total.Backward.Packets       int64  
 9   Total.Length.of.Fwd.Packets  int64  
 10  Total.Length.of.Bwd.Packets  float64
 11  Fwd.Packet.Length.Max        int64  
 12  Fwd.Packet.Length.Min        int64  
 13  Fwd.Packet.Length.Mean       float64
 14  Fwd.Packet.Length.Std        float64
 15  Bwd.Packet.Length.Max        int64  
 16  Bwd.Packet.Length.Min        int64  
 17  Bwd.Packet.Length.Mean       float64
 18  Bwd.Packet.Length.Std        float64
 19  

### Describe the dataset

In [41]:
network_traffic_analysis_dataframe.iloc[:,:10].describe()

,Source.Port,Destination.Port,Protocol,Flow.Duration,Total.Fwd.Packets,Total.Backward.Packets,Total.Length.of.Fwd.Packets
count,3.577296e+06,3.577296e+06,3.577296e+06,3.577296e+06,3.577296e+06,3.577296e+06,3.577296e+06
mean,3.799938e+04,1.204246e+04,6.005508e+00,2.544247e+07,6.237799e+01,6.534083e+01,4.683323e+04
std,2.201713e+04,2.044916e+04,3.274574e-01,4.014430e+07,1.094086e+03,1.108092e+03,1.816196e+06
min,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,3.697000e+03,4.430000e+02,6.000000e+00,6.280000e+02,2.000000e+00,1.000000e+00,1.200000e+01
50%,4.937700e+04,3.128000e+03,6.000000e+00,5.847295e+05,6.000000e+00,5.000000e+00,4.430000e+02
75%,5.379900e+04,3.128000e+03,6.000000e+00,4.500153e+07,1.500000e+01,1.500000e+01,1.769000e+03
max,6.553400e+04,6.553400e+04,1.700000e+01,1.200000e+08,4.531900e+05,5.421960e+05,6.780236e+08


In [42]:
network_traffic_analysis_dataframe.iloc[:,11:20].describe()

,Fwd.Packet.Length.Max,Fwd.Packet.Length.Min,Fwd.Packet.Length.Mean,Fwd.Packet.Length.Std,Bwd.Packet.Length.Max,Bwd.Packet.Length.Min,Bwd.Packet.Length.Mean,Bwd.Packet.Length.Std,Flow.Bytes.s
count,3.577296e+06,3.577296e+06,3.577296e+06,3.577296e+06,3.577296e+06,3.577296e+06,3.577296e+06,3.577296e+06,3.577296e+06
mean,5.123645e+02,9.340408e+00,1.149212e+02,1.520501e+02,1.103231e+03,1.113491e+01,2.547845e+02,2.898878e+02,4.048709e+06
std,1.039319e+03,8.299983e+01,2.464707e+02,2.404702e+02,2.352374e+03,1.055422e+02,5.060731e+02,4.853004e+02,7.551040e+07
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,6.000000e+00,0.000000e+00,6.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.882429e+01
50%,2.060000e+02,0.000000e+00,4.657143e+01,7.421124e+01,8.100000e+01,0.000000e+00,3.014286e+01,3.242474e+01,1.140944e+03
75%,6.130000e+02,6.000000e+00,1.225000e+02,2.079035e+02,1.366000e+03,0.000000e+00,2.567500e+02,4.232105e+02,2.343750e+04
max,3.283200e+04,1.606000e+04,1.606000e+04,6.225487e+03,3.764800e+04,1.303200e+04,1.303200e+04,8.434804e+03,1.439600e+10


In [43]:
network_traffic_analysis_dataframe.iloc[:,21:30].describe()

,Flow.IAT.Mean,Flow.IAT.Std,Flow.IAT.Max,Flow.IAT.Min,Fwd.IAT.Total,Fwd.IAT.Mean,Fwd.IAT.Std,Fwd.IAT.Max,Fwd.IAT.Min
count,3.577296e+06,3.577296e+06,3.577296e+06,3.577296e+06,3.577296e+06,3.577296e+06,3.577296e+06,3.577296e+06,3.577296e+06
mean,1.422201e+06,3.365395e+06,1.285020e+07,8.870201e+04,2.418796e+07,3.124467e+06,3.649620e+06,1.209624e+07,1.271532e+06
std,3.550414e+06,6.260959e+06,2.076518e+07,1.605272e+06,3.962563e+07,8.358652e+06,7.390979e+06,2.049180e+07,7.279117e+06
min,2.000000e-01,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.150000e+02,8.485281e+00,5.700000e+02,0.000000e+00,7.000000e+00,5.000000e+00,0.000000e+00,6.000000e+00,0.000000e+00
50%,3.320238e+04,6.836444e+04,2.812395e+05,1.000000e+00,3.892645e+05,3.700679e+04,4.717596e+04,2.076290e+05,0.000000e+00
75%,9.366576e+05,3.980748e+06,2.391546e+07,3.300000e+01,4.001161e+07,1.549711e+06,2.932647e+06,1.926976e+07,9.200000e+01
max,1.200000e+08,8.485273e+07,1.200000e+08,1.200000e+08,1.200000e+08,1.200000e+08,8.485256e+07,1.200000e+08,1.200000e+08


In [44]:
network_traffic_analysis_dataframe.iloc[:,31:40].describe()

,Bwd.IAT.Mean,Bwd.IAT.Std,Bwd.IAT.Max,Bwd.IAT.Min,Fwd.PSH.Flags,Bwd.PSH.Flags,Fwd.URG.Flags,Bwd.URG.Flags,Fwd.Header.Length
count,3.577296e+06,3.577296e+06,3.577296e+06,3.577296e+06,3.577296e+06,3577296.0,3577296.0,3577296.0,3.577296e+06
mean,2.476877e+06,2.932460e+06,9.830803e+06,8.889991e+05,1.720414e-01,0.0,0.0,0.0,1.653339e+03
std,7.578111e+06,6.666650e+06,1.883521e+07,6.231082e+06,3.774165e-01,0.0,0.0,0.0,3.008890e+04
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,4.000000e+01
50%,1.558765e+04,2.617595e+04,9.518150e+04,0.000000e+00,0.000000e+00,0.0,0.0,0.0,1.520000e+02
75%,3.342142e+05,7.526342e+05,7.508778e+06,1.000000e+00,0.000000e+00,0.0,0.0,0.0,3.920000e+02
max,1.199999e+08,8.485275e+07,1.199999e+08,1.199999e+08,1.000000e+00,0.0,0.0,0.0,1.543950e+07


In [45]:
network_traffic_analysis_dataframe.iloc[:,41:50].describe()

,Fwd.Packets.s,Bwd.Packets.s,Min.Packet.Length,Max.Packet.Length,Packet.Length.Mean,Packet.Length.Std,Packet.Length.Variance,FIN.Flag.Count,SYN.Flag.Count
count,3.577296e+06,3.577296e+06,3.577296e+06,3.577296e+06,3.577296e+06,3.577296e+06,3.577296e+06,3.577296e+06,3.577296e+06
mean,7.705816e+04,1.190522e+04,3.043745e+00,1.333250e+03,1.988191e+02,3.035190e+02,2.792736e+05,7.037159e-03,1.720414e-01
std,3.683153e+05,1.080206e+05,4.145472e+01,2.453395e+03,3.327427e+02,4.326083e+02,7.258608e+05,8.359210e-02,3.774165e-01
min,8.333337e-03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.417242e-01,1.009873e-01,0.000000e+00,6.000000e+00,6.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.563422e+01,2.951696e+00,0.000000e+00,3.550000e+02,6.283333e+01,1.069828e+02,1.144531e+04,0.000000e+00,0.000000e+00
75%,2.164502e+03,8.344459e+01,6.000000e+00,1.460000e+03,2.500000e+02,4.818125e+02,2.321432e+05,0.000000e+00,0.000000e+00
max,6.000000e+06,5.000000e+06,7.063000e+03,3.764800e+04,1.070867e+04,9.268781e+03,8.591031e+07,1.000000e+00,1.000000e+00


In [46]:
network_traffic_analysis_dataframe.iloc[:,51:60].describe()

,PSH.Flag.Count,ACK.Flag.Count,URG.Flag.Count,CWE.Flag.Count,ECE.Flag.Count,Down.Up.Ratio,Average.Packet.Size,Avg.Fwd.Segment.Size,Avg.Bwd.Segment.Size
count,3.577296e+06,3.577296e+06,3.577296e+06,3577296.0,3.577296e+06,3.577296e+06,3.577296e+06,3.577296e+06,3.577296e+06
mean,4.058210e-01,5.995705e-01,2.773847e-01,0.0,6.566412e-04,9.085471e-01,2.075630e+02,1.149212e+02,2.547845e+02
std,4.910503e-01,4.899855e-01,4.477080e-01,0.0,2.561660e-02,1.269945e+00,3.432270e+02,2.464707e+02,5.060731e+02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,9.000000e+00,6.000000e+00,0.000000e+00
50%,0.000000e+00,1.000000e+00,0.000000e+00,0.0,0.000000e+00,1.000000e+00,6.650000e+01,4.657143e+01,3.014286e+01
75%,1.000000e+00,1.000000e+00,1.000000e+00,0.0,0.000000e+00,1.000000e+00,2.637184e+02,1.225000e+02,2.567500e+02
max,1.000000e+00,1.000000e+00,1.000000e+00,0.0,1.000000e+00,2.930000e+02,1.606300e+04,1.606000e+04,1.303200e+04


In [47]:
network_traffic_analysis_dataframe.iloc[:,61:70].describe()

,Fwd.Avg.Bytes.Bulk,Fwd.Avg.Packets.Bulk,Fwd.Avg.Bulk.Rate,Bwd.Avg.Bytes.Bulk,Bwd.Avg.Packets.Bulk,Bwd.Avg.Bulk.Rate,Subflow.Fwd.Packets,Subflow.Fwd.Bytes,Subflow.Bwd.Packets
count,3577296.0,3577296.0,3577296.0,3577296.0,3577296.0,3577296.0,3.577296e+06,3.577296e+06,3.577296e+06
mean,0.0,0.0,0.0,0.0,0.0,0.0,6.237799e+01,4.683323e+04,6.534083e+01
std,0.0,0.0,0.0,0.0,0.0,0.0,1.094086e+03,1.816196e+06,1.108092e+03
min,0.0,0.0,0.0,0.0,0.0,0.0,1.000000e+00,0.000000e+00,0.000000e+00
25%,0.0,0.0,0.0,0.0,0.0,0.0,2.000000e+00,1.200000e+01,1.000000e+00
50%,0.0,0.0,0.0,0.0,0.0,0.0,6.000000e+00,4.430000e+02,5.000000e+00
75%,0.0,0.0,0.0,0.0,0.0,0.0,1.500000e+01,1.769000e+03,1.500000e+01
max,0.0,0.0,0.0,0.0,0.0,0.0,4.531900e+05,6.780236e+08,5.421960e+05


In [50]:
network_traffic_analysis_dataframe.iloc[:,71:80].describe()

,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active.Mean,Active.Std,Active.Max,Active.Min,Idle.Mean
count,3.577296e+06,3.577296e+06,3.577296e+06,3.577296e+06,3.577296e+06,3.577296e+06,3.577296e+06,3.577296e+06,3.577296e+06
mean,8.984691e+03,2.123489e+03,4.503535e+01,2.569738e+01,2.981990e+05,1.836406e+05,5.229372e+05,1.676336e+05,8.524211e+06
std,1.410126e+04,7.704789e+03,9.748192e+02,6.025989e+00,2.349390e+06,1.325838e+06,3.266508e+06,2.064219e+06,1.706568e+07
min,-1.000000e+00,-1.000000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.110000e+02,1.800000e+01,0.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,5.840000e+03,2.620000e+02,2.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.460000e+04,6.600000e+02,9.000000e+00,3.200000e+01,4.500000e+01,0.000000e+00,5.700000e+01,2.000000e+00,7.506747e+06
max,6.553500e+04,6.553500e+04,3.286940e+05,5.230000e+02,1.146950e+08,7.297136e+07,1.146950e+08,1.146950e+08,1.200000e+08


In [51]:
network_traffic_analysis_dataframe.iloc[:,81:].describe()

,Idle.Max,Idle.Min,L7Protocol
count,3.577296e+06,3.577296e+06,3.577296e+06
mean,9.743845e+06,7.252097e+06,1.029508e+02
std,1.888557e+07,1.600754e+07,5.129198e+01
min,0.000000e+00,0.000000e+00,1.000000e+00
25%,0.000000e+00,0.000000e+00,9.100000e+01
50%,0.000000e+00,0.000000e+00,1.260000e+02
75%,8.034389e+06,5.369712e+06,1.300000e+02
max,1.200000e+08,1.200000e+08,2.220000e+02


### Detect for any null values in the dataset

In [55]:
network_traffic_analysis_dataframe.isnull().sum()[:20]

Source.IP                      0
Source.Port                    0
Destination.IP                 0
Destination.Port               0
Protocol                       0
Timestamp                      0
Flow.Duration                  0
Total.Fwd.Packets              0
Total.Backward.Packets         0
Total.Length.of.Fwd.Packets    0
Total.Length.of.Bwd.Packets    0
Fwd.Packet.Length.Max          0
Fwd.Packet.Length.Min          0
Fwd.Packet.Length.Mean         0
Fwd.Packet.Length.Std          0
Bwd.Packet.Length.Max          0
Bwd.Packet.Length.Min          0
Bwd.Packet.Length.Mean         0
Bwd.Packet.Length.Std          0
Flow.Bytes.s                   0
dtype: int64

In [56]:
network_traffic_analysis_dataframe.isnull().sum()[21:40]

Flow.IAT.Mean        0
Flow.IAT.Std         0
Flow.IAT.Max         0
Flow.IAT.Min         0
Fwd.IAT.Total        0
Fwd.IAT.Mean         0
Fwd.IAT.Std          0
Fwd.IAT.Max          0
Fwd.IAT.Min          0
Bwd.IAT.Total        0
Bwd.IAT.Mean         0
Bwd.IAT.Std          0
Bwd.IAT.Max          0
Bwd.IAT.Min          0
Fwd.PSH.Flags        0
Bwd.PSH.Flags        0
Fwd.URG.Flags        0
Bwd.URG.Flags        0
Fwd.Header.Length    0
dtype: int64

In [57]:
network_traffic_analysis_dataframe.isnull().sum()[41:60]

Fwd.Packets.s             0
Bwd.Packets.s             0
Min.Packet.Length         0
Max.Packet.Length         0
Packet.Length.Mean        0
Packet.Length.Std         0
Packet.Length.Variance    0
FIN.Flag.Count            0
SYN.Flag.Count            0
RST.Flag.Count            0
PSH.Flag.Count            0
ACK.Flag.Count            0
URG.Flag.Count            0
CWE.Flag.Count            0
ECE.Flag.Count            0
Down.Up.Ratio             0
Average.Packet.Size       0
Avg.Fwd.Segment.Size      0
Avg.Bwd.Segment.Size      0
dtype: int64

In [58]:
network_traffic_analysis_dataframe.isnull().sum()[61:80]

Fwd.Avg.Bytes.Bulk         0
Fwd.Avg.Packets.Bulk       0
Fwd.Avg.Bulk.Rate          0
Bwd.Avg.Bytes.Bulk         0
Bwd.Avg.Packets.Bulk       0
Bwd.Avg.Bulk.Rate          0
Subflow.Fwd.Packets        0
Subflow.Fwd.Bytes          0
Subflow.Bwd.Packets        0
Subflow.Bwd.Bytes          0
Init_Win_bytes_forward     0
Init_Win_bytes_backward    0
act_data_pkt_fwd           0
min_seg_size_forward       0
Active.Mean                0
Active.Std                 0
Active.Max                 0
Active.Min                 0
Idle.Mean                  0
dtype: int64

In [59]:
network_traffic_analysis_dataframe.isnull().sum()[81:]

Idle.Max        0
Idle.Min        0
Label           0
L7Protocol      0
ProtocolName    0
dtype: int64

### Detect for any na values in the dataset

In [61]:
network_traffic_analysis_dataframe.isna().sum()[:20]

Source.IP                      0
Source.Port                    0
Destination.IP                 0
Destination.Port               0
Protocol                       0
Timestamp                      0
Flow.Duration                  0
Total.Fwd.Packets              0
Total.Backward.Packets         0
Total.Length.of.Fwd.Packets    0
Total.Length.of.Bwd.Packets    0
Fwd.Packet.Length.Max          0
Fwd.Packet.Length.Min          0
Fwd.Packet.Length.Mean         0
Fwd.Packet.Length.Std          0
Bwd.Packet.Length.Max          0
Bwd.Packet.Length.Min          0
Bwd.Packet.Length.Mean         0
Bwd.Packet.Length.Std          0
Flow.Bytes.s                   0
dtype: int64

In [62]:
network_traffic_analysis_dataframe.isna().sum()[21:40]

Flow.IAT.Mean        0
Flow.IAT.Std         0
Flow.IAT.Max         0
Flow.IAT.Min         0
Fwd.IAT.Total        0
Fwd.IAT.Mean         0
Fwd.IAT.Std          0
Fwd.IAT.Max          0
Fwd.IAT.Min          0
Bwd.IAT.Total        0
Bwd.IAT.Mean         0
Bwd.IAT.Std          0
Bwd.IAT.Max          0
Bwd.IAT.Min          0
Fwd.PSH.Flags        0
Bwd.PSH.Flags        0
Fwd.URG.Flags        0
Bwd.URG.Flags        0
Fwd.Header.Length    0
dtype: int64

In [63]:
network_traffic_analysis_dataframe.isna().sum()[41:60]

Fwd.Packets.s             0
Bwd.Packets.s             0
Min.Packet.Length         0
Max.Packet.Length         0
Packet.Length.Mean        0
Packet.Length.Std         0
Packet.Length.Variance    0
FIN.Flag.Count            0
SYN.Flag.Count            0
RST.Flag.Count            0
PSH.Flag.Count            0
ACK.Flag.Count            0
URG.Flag.Count            0
CWE.Flag.Count            0
ECE.Flag.Count            0
Down.Up.Ratio             0
Average.Packet.Size       0
Avg.Fwd.Segment.Size      0
Avg.Bwd.Segment.Size      0
dtype: int64

In [64]:
network_traffic_analysis_dataframe.isna().sum()[61:80]

Fwd.Avg.Bytes.Bulk         0
Fwd.Avg.Packets.Bulk       0
Fwd.Avg.Bulk.Rate          0
Bwd.Avg.Bytes.Bulk         0
Bwd.Avg.Packets.Bulk       0
Bwd.Avg.Bulk.Rate          0
Subflow.Fwd.Packets        0
Subflow.Fwd.Bytes          0
Subflow.Bwd.Packets        0
Subflow.Bwd.Bytes          0
Init_Win_bytes_forward     0
Init_Win_bytes_backward    0
act_data_pkt_fwd           0
min_seg_size_forward       0
Active.Mean                0
Active.Std                 0
Active.Max                 0
Active.Min                 0
Idle.Mean                  0
dtype: int64

In [65]:
network_traffic_analysis_dataframe.isna().sum()[81:]

Idle.Max        0
Idle.Min        0
Label           0
L7Protocol      0
ProtocolName    0
dtype: int64

### Determine the classes in data columns that are object data type

In [67]:
network_traffic_analysis_dataframe.select_dtypes("object")

,Source.IP,Destination.IP,Timestamp,Label,ProtocolName
0,172.19.1.46,10.200.7.7,26/04/201711:11:17,BENIGN,HTTP_PROXY
1,10.200.7.7,172.19.1.46,26/04/201711:11:17,BENIGN,HTTP_PROXY
2,50.31.185.39,10.200.7.217,26/04/201711:11:17,BENIGN,HTTP
3,50.31.185.39,10.200.7.217,26/04/201711:11:17,BENIGN,HTTP
4,192.168.72.43,10.200.7.7,26/04/201711:11:17,BENIGN,HTTP_PROXY
...,...,...,...,...,...
3577291,98.138.79.73,10.200.7.199,15/05/201705:43:40,BENIGN,SSL
3577292,98.138.79.73,10.200.7.217,15/05/201705:46:10,BENIGN,SSL
3577293,98.138.79.73,10.200.7.218,15/05/201705:45:39,BENIGN,SSL
3577294,98.138.79.73,10.200.7.195,15/05/201705:45:59,BENIGN,SSL


In [68]:
network_traffic_analysis_dataframe["Label"].unique()

array(['BENIGN'], dtype=object)

In [70]:
network_traffic_analysis_dataframe["L7Protocol"].unique()

array([131,   7, 130,  91, 126, 124, 119,  40, 121, 147, 178, 212, 163,
       122,  70,  68,  64, 125, 221, 114, 120, 143, 220, 169, 219, 176,
       201,   5,  60, 142, 145, 175, 132, 140, 222, 211, 179, 123,  81,
         9, 148, 156, 203,  51, 195, 133,  92, 200,  67, 135, 153,  36,
        69, 167, 210, 159, 170, 164, 213,  11, 174, 162,  14, 202,  48,
       185,   1, 150, 158, 139, 134,  85, 180,  13, 146, 172,  37, 191])

In [74]:
network_traffic_analysis_dataframe[network_traffic_analysis_dataframe["ProtocolName"] == "HTTP_CONNECT"].head(1)

,Source.IP,Source.Port,Destination.IP,Destination.Port,Protocol,Timestamp,Flow.Duration,Total.Fwd.Packets,Total.Backward.Packets,Total.Length.of.Fwd.Packets,...,Active.Std,Active.Max,Active.Min,Idle.Mean,Idle.Std,Idle.Max,Idle.Min,Label,L7Protocol,ProtocolName
7,192.168.10.47,51848,10.200.7.6,3128,6,26/04/201711:11:17,11002,3,12,232,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,130,HTTP_CONNECT


In [69]:
network_traffic_analysis_dataframe["ProtocolName"].unique()

array(['HTTP_PROXY', 'HTTP', 'HTTP_CONNECT', 'SSL', 'GOOGLE', 'YOUTUBE',
       'FACEBOOK', 'CONTENT_FLASH', 'DROPBOX', 'WINDOWS_UPDATE', 'AMAZON',
       'MICROSOFT', 'TOR', 'GMAIL', 'YAHOO', 'MSN', 'SSL_NO_CERT',
       'SKYPE', 'MS_ONE_DRIVE', 'MSSQL', 'TWITTER', 'APPLE_ICLOUD',
       'CLOUDFLARE', 'UBUNTUONE', 'OFFICE_365', 'WIKIPEDIA', 'OPENSIGNAL',
       'DNS', 'HTTP_DOWNLOAD', 'WHATSAPP', 'APPLE_ITUNES', 'FTP_DATA',
       'CITRIX', 'APPLE', 'MQTT', 'INSTAGRAM', 'EBAY', 'GOOGLE_MAPS',
       'IP_ICMP', 'NTP', 'TEAMVIEWER', 'SPOTIFY', 'EASYTAXI',
       'MAIL_IMAPS', 'TWITCH', 'NETFLIX', 'SSH', 'SIMET',
       'UNENCRYPED_JABBER', 'WAZE', 'UPNP', 'EDONKEY', 'OSCAR', 'ORACLE',
       'DEEZER', 'OPENVPN', 'WHOIS_DAS', 'SKINNY', 'STARCRAFT', 'NFS',
       'RTMP', 'TEAMSPEAK', 'SNMP', '99TAXI', 'QQ', 'TELEGRAM',
       'FTP_CONTROL', 'LOTUS_NOTES', 'H323', 'CITRIX_ONLINE', 'LASTFM',
       'IP_OSPF', 'CNN', 'BGP', 'RADIUS', 'SOCKS', 'BITTORRENT', 'TIMMEU'],
      dtype=object)

### Mapping of L7Protocol and ProtocolName

In [ ]:
# Create a dictionary based on
## - network_traffic_analysis_dataframe["L7Protocol"].unique()
## - network_traffic_analysis_dataframe["ProtocolName"].unique()

### Mapping of Protocol and OSI Model

In [ ]:
# Create a mapping of the number in Protocol column and the OSI model

### Find a method to create new records of MALIGNANT data and put it into the dataset

In [ ]:
#TODO

### Split TimeStamp into 2 different columns (Date and Time)

In [ ]:
#TODO

# Data Preparation for ML Training

* Load and explore the dataset.
* Handle missing data and outliers.
* Perform data visualization to gain insights into the dataset.
* Preprocess the data for modeling, including feature scaling and encoding categorical variables.

### Remove Flow.ID column

This is because this column Flow.ID is an identifier for each row. There is no meaning in the data therefore it should be removed.

In [ ]:
network_traffic_analysis_dataframe.drop(labels="Flow.ID", axis=1, inplace=True)

### TEST

In [66]:
network_traffic_analysis_dataframe

,Source.IP,Source.Port,Destination.IP,Destination.Port,Protocol,Timestamp,Flow.Duration,Total.Fwd.Packets,Total.Backward.Packets,Total.Length.of.Fwd.Packets,...,Active.Std,Active.Max,Active.Min,Idle.Mean,Idle.Std,Idle.Max,Idle.Min,Label,L7Protocol,ProtocolName
0,172.19.1.46,52422,10.200.7.7,3128,6,26/04/201711:11:17,45523,22,55,132,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,131,HTTP_PROXY
1,10.200.7.7,3128,172.19.1.46,52422,6,26/04/201711:11:17,1,2,0,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,131,HTTP_PROXY
2,50.31.185.39,80,10.200.7.217,38848,6,26/04/201711:11:17,1,3,0,674,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,7,HTTP
3,50.31.185.39,80,10.200.7.217,38848,6,26/04/201711:11:17,217,1,3,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,7,HTTP
4,192.168.72.43,55961,10.200.7.7,3128,6,26/04/201711:11:17,78068,5,0,1076,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,131,HTTP_PROXY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3577291,98.138.79.73,443,10.200.7.199,42135,6,15/05/201705:43:40,2290821,5,4,599,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,91,SSL
3577292,98.138.79.73,443,10.200.7.217,51546,6,15/05/201705:46:10,24,5,0,1448,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,91,SSL
3577293,98.138.79.73,443,10.200.7.218,44366,6,15/05/201705:45:39,2591653,6,5,1202,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,91,SSL
3577294,98.138.79.73,443,10.200.7.195,52341,6,15/05/201705:45:59,2622421,4,3,632,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,91,SSL


In [ ]:
network_traffic_analysis_dataframe[]

# Model Building

* Split the dataset into training and testing sets.
* Implement at least three different classification models (e.g., Decision Tree, Random Forest, SVM, etc.).
* Train and fine-tune each model using appropriate techniques.
* Discuss the choice of hyperparameters and the reasoning behind it

# Model Evaluation

* Evaluate the models using appropriate classification metrics (accuracy, precision, recall, F1-score, etc.).
* Visualize the model performance using ROC curves and confusion matrices.
* Compare the models and justify your choice of the best-performing model.

# Recommendation and Action Plan

# Conclusion

# Reference